In [1]:
# !pip install konlpy

In [2]:
import re
from konlpy.tag import Okt
from collections import Counter
print("임포트 완료")

임포트 완료


In [3]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

## 영어 Word2Vec 실습과 OOV 문제

영어 Word2Vec 실습
----

영어 데이터를 다운로드받아 직접 Word2Vec을 훈련시켜보겠습니다. Word2Vec을 별도로 구현할 필요없이 파이썬의 gensim 패키지를 통해 이미 구현된 Word2Vec 모델을 사용할 수 있습니다. 여기서 사용할 훈련 데이터는 NLTK에서 제공하는 코퍼스이며, gensim 패키지는 토픽 모델링을 위한 NLP 패키지입니다.

Cloud 사용자는 이미 설치되어 있으니 아래의 명령어를 참고로만 알아두세요.

    pip install nltk
    pip install gensim

In [5]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /Users/seongyeon/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /Users/seongyeon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from nltk.corpus import abc
corpus = abc.sents()
print("슝~")

슝~


In [7]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [8]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [9]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("모델 학습 완료!")

모델 학습 완료!


위 코드에서 각 파라미터가 의미하는 바는 아래와 같습니다.

- vector size = 학습 후 임베딩 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않아요.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBoW, 1은 Skip-gram.

아주 잠깐의 기다림 끝에 Word2Vec의 학습이 완료됩니다. Word2Vec는 입력한 단어에 대해서 가장 코사인 유사도가 높은 단어들을 출력하는 model.wv.most_similar를 지원합니다. 'man'과 가장 유사한 단어들은 어떤 단어들일까요?

In [10]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9232221841812134), ('skull', 0.9111101627349854), ('Bang', 0.9060567021369934), ('asteroid', 0.9052808284759521), ('third', 0.9019376039505005), ('baby', 0.8993116021156311), ('dog', 0.8986425399780273), ('bought', 0.8975557088851929), ('rally', 0.8911317586898804), ('disc', 0.8890964388847351)]


In [12]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('./w2v') 
loaded_model = KeyedVectors.load_word2vec_format("./w2v")
print("모델  load 완료!")

모델  load 완료!


## FastText
- subword 단위로 쪼개서 오타에 강건함

In [13]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)
print("FastText 학습 완료!")

FastText 학습 완료!


In [14]:
fasttext_model.wv.most_similar('overacting')

[('overwhelming', 0.9444938898086548),
 ('extracting', 0.9408138990402222),
 ('lifting', 0.9358069896697998),
 ('shooting', 0.932708740234375),
 ('tapping', 0.9303584098815918),
 ('attracting', 0.9299964904785156),
 ('contracting', 0.9287461638450623),
 ('shifting', 0.9276260137557983),
 ('emptying', 0.9275338649749756),
 ('clotting', 0.9267818331718445)]

In [15]:
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.9551022052764893),
 ('musical', 0.8601377606391907),
 ('emotion', 0.8544632196426392),
 ('performance', 0.8536600470542908),
 ('intelligence', 0.8513544797897339),
 ('emotional', 0.8466050624847412),
 ('mess', 0.8460673689842224),
 ('flexibility', 0.8446962833404541),
 ('visual', 0.8442155122756958),
 ('cognitive', 0.8404098749160767)]

## Glove 실습

In [16]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-50")  # glove vectors 다운로드
glove_model.most_similar("dog")  # 'dog'과 비슷한 단어 찾기

[==================================================] 100.0% 66.0/66.0MB downloaded


[('cat', 0.9218006134033203),
 ('dogs', 0.8513158559799194),
 ('horse', 0.7907583117485046),
 ('puppy', 0.7754921317100525),
 ('pet', 0.7724707722663879),
 ('rabbit', 0.7720814347267151),
 ('pig', 0.7490062117576599),
 ('snake', 0.7399188876152039),
 ('baby', 0.7395570874214172),
 ('bite', 0.738793671131134)]

In [17]:
glove_model.most_similar('overacting')

[('impudence', 0.7842012047767639),
 ('puerile', 0.7816032767295837),
 ('winningly', 0.7644237279891968),
 ('grossness', 0.7576098442077637),
 ('deconstructions', 0.748936653137207),
 ('over-the-top', 0.7460805773735046),
 ('buffoonery', 0.746045708656311),
 ('impetuosity', 0.7415392994880676),
 ('sophomoric', 0.736961841583252),
 ('zaniness', 0.7353197336196899)]

In [18]:
glove_model.most_similar('memoryy')

KeyError: "Key 'memoryy' not present in vocabulary"

GloVe는 Word2Vec과 같이 OOV 문제를 가지고 있어서 'memoryy'라는 단어는 인식하지 못합니다. 또한 pre-trained GloVe 모델은 한글이나 알파벳 대문자가 포함된 데이터셋으로 학습하지 않았기 때문에 알파벳 소문자만 인식한다는 사실에 유의하세요!